In [2]:
import os
import time
import datetime
import requests
import pandas as pd
import pyarrow
import numpy as np
from prefect import flow, task # Prefect flow and task decorators
import time
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

In [3]:
#read firms_output parquet
df_firms = pd.read_parquet("firms_outout", engine="pyarrow")
df_firms.info()
df_firms

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3280 entries, 0 to 3279
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype                       
---  ------           --------------  -----                       
 0   country_id       3280 non-null   object                      
 1   latitude         3280 non-null   float64                     
 2   longitude        3280 non-null   float64                     
 3   brightness       3280 non-null   float64                     
 4   scan             3280 non-null   float64                     
 5   track            3280 non-null   float64                     
 6   acq_date         3280 non-null   object                      
 7   acq_time         3280 non-null   object                      
 8   satellite        3280 non-null   object                      
 9   instrument       3280 non-null   object                      
 10  confidence       3280 non-null   int64                       
 11  version          

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,297.41,28.27,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
1,THA,19.45886,100.53445,316.00,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,298.49,43.84,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
2,THA,19.46471,100.54360,315.08,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,298.54,39.24,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
3,THA,19.47333,98.03783,309.05,1.75,1.30,2025-03-28,03:28:00,Terra,MODIS,...,296.34,12.68,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
4,THA,14.07963,100.65222,321.68,3.57,1.77,2025-03-28,03:30:00,Terra,MODIS,...,297.11,64.02,D,2025-03-28 03:30:00,2025-03-28 10:30:00+07:00,2025,3,28,10,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,THA,18.07610,103.97359,316.76,1.15,1.07,2025-05-04,14:10:00.000000,Terra,MODIS,...,292.40,14.99,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3276,THA,19.70258,98.09076,302.77,2.76,1.59,2025-05-04,14:10:00.000000,Terra,MODIS,...,291.74,16.23,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3277,THA,17.54023,99.73300,302.68,1.75,1.30,2025-05-03,20:29:00.000000,Aqua,MODIS,...,290.33,7.54,N,2025-05-03 20:29:00,2025-05-04 03:29:00+07:00,2025,5,4,3,29
3278,THA,17.54023,99.73300,302.68,1.75,1.30,2025-05-03,20:29:00.000000,Aqua,MODIS,...,290.33,7.54,N,2025-05-03 20:29:00,2025-05-04 03:29:00+07:00,2025,5,4,3,29


In [4]:
#read weather_output parquet
df_weather = pd.read_parquet("weather_outout", engine="pyarrow")
df_weather.info()
df_weather

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4004 entries, 0 to 4003
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   coord.lon            4004 non-null   float64                     
 1   coord.lat            4004 non-null   float64                     
 2   weather.id           4004 non-null   int64                       
 3   weather.main         4004 non-null   object                      
 4   weather.description  4004 non-null   object                      
 5   weather.icon         4004 non-null   object                      
 6   base                 4004 non-null   object                      
 7   main.temp            4004 non-null   float64                     
 8   main.feels_like      4004 non-null   float64                     
 9   main.pressure        4004 non-null   int64                       
 10  main.humidity        4004 non-null  

,coord.lon,coord.lat,weather.id,weather.main,weather.description,weather.icon,base,main.temp,main.feels_like,main.pressure,...,sys.sunset,timezone,id,name,cod,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,100.5241,13.7279,803,Clouds,broken clouds,04n,stations,302.50,307.95,1004,...,1743161353,25200,1619278,Bangrak,200,2025,3,28,1,6
1,98.9063,8.0863,804,Clouds,overcast clouds,04n,stations,298.67,299.47,1007,...,1743161665,25200,1152633,Krabi,200,2025,3,28,1,6
2,99.5328,14.0228,802,Clouds,scattered clouds,03n,stations,301.23,302.61,1004,...,1743161595,25200,1157835,Ban Don Rak,200,2025,3,28,1,6
3,103.5059,16.4314,804,Clouds,overcast clouds,04n,stations,304.73,305.54,1002,...,1743160676,25200,1610469,Kalasin,200,2025,3,28,1,6
4,99.5227,16.4828,802,Clouds,scattered clouds,03n,stations,299.77,299.77,1003,...,1743161633,25200,1153090,Kamphaeng Phet,200,2025,3,28,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,102.7872,17.4138,803,Clouds,broken clouds,04d,stations,311.75,318.75,1006,...,1746444606,25200,1605239,Udon Thani,200,2025,5,5,13,7
4000,100.0993,17.6201,802,Clouds,scattered clouds,03d,stations,308.58,311.84,1005,...,1746445268,25200,1605215,Uttaradit,200,2025,5,5,13,7
4001,100.0246,15.3835,804,Clouds,overcast clouds,04d,stations,308.71,311.76,1005,...,1746445112,25200,1605221,Uthai Thani,200,2025,5,5,13,7
4002,104.8564,15.2287,804,Clouds,overcast clouds,04d,stations,307.29,310.85,1006,...,1746443941,25200,1605245,Ubon Ratchathani,200,2025,5,5,13,7


In [6]:
df_weather = df_weather.rename(columns={'coord.lat': 'latitude', 'coord.lon': 'longitude'})

In [7]:
import pandas as pd
import numpy as np

from sklearn.neighbors import BallTree
from io import StringIO

import matplotlib.pyplot as plt
import contextily as ctx

find closest location

In [8]:
# Convert coordinates to radians
firms_coords = np.deg2rad(df_firms[['latitude', 'longitude']])
weather_coords = np.deg2rad(df_weather[['latitude', 'longitude']])

# Create a BallTree for nearest neighbor search
tree = BallTree(weather_coords, metric='haversine')

# Find the nearest neighbors in OSM data
distances, indices = tree.query(firms_coords, k=1)

test_df = pd.DataFrame()
# Add the results to the DataFrame
df_firms['nearest_index'] = indices.flatten()
df_firms['distance_meters'] = distances.flatten() * 6371000 # Earth's radius in meters

# Extract and assign the matched OSM node details
df_firms[[
    'near_lat',
    'near_lon',
]] = df_weather.iloc[df_firms['nearest_index']][[
    'latitude',
    'longitude',
]].values

# test_df['node_id'] = test_df['node_id'].apply(lambda x: f'{int(x)}')
# test_df = test_df.drop('nearest_index', axis=1)

df_firms.head(5)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute,nearest_index,distance_meters,near_lat,near_lon
0,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,2025-03-28 10:28:00+07:00,2025,3,28,10,28,33,46033.117171,19.2149,100.2024
1,THA,19.45886,100.53445,316.00,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,2025-03-28 10:28:00+07:00,2025,3,28,10,28,33,44154.981405,19.2149,100.2024
2,THA,19.46471,100.54360,315.08,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,2025-03-28 10:28:00+07:00,2025,3,28,10,28,33,45311.666783,19.2149,100.2024
3,THA,19.47333,98.03783,309.05,1.75,1.30,2025-03-28,03:28:00,Terra,MODIS,...,2025-03-28 10:28:00+07:00,2025,3,28,10,28,1584,20509.954218,19.3020,97.9654
4,THA,14.07963,100.65222,321.68,3.57,1.77,2025-03-28,03:30:00,Terra,MODIS,...,2025-03-28 10:30:00+07:00,2025,3,28,10,30,386,14352.531111,14.0204,100.5340


merge to nearest datetime

In [9]:
df_firms = df_firms.sort_values('acq_datetime_th')
df_weather = df_weather.sort_values('dt')

In [39]:
dfmerge = pd.merge_asof(df_firms, df_weather,
...                         left_on='acq_datetime_th' , right_on='dt',
                            left_by=['near_lat','near_lon'], right_by=['latitude','longitude'],
                            direction='nearest')

In [33]:
dfmerge.info()
dfmerge.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3280 entries, 0 to 3279
Data columns (total 65 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   country_id           3280 non-null   object                      
 1   latitude_x           3280 non-null   float64                     
 2   longitude_x          3280 non-null   float64                     
 3   brightness           3280 non-null   float64                     
 4   scan                 3280 non-null   float64                     
 5   track                3280 non-null   float64                     
 6   acq_date_x           3280 non-null   object                      
 7   acq_time_x           3280 non-null   object                      
 8   satellite            3280 non-null   object                      
 9   instrument           3280 non-null   object                      
 10  confidence           3280 non-null  

,country_id,latitude_x,longitude_x,brightness,scan,track,acq_date_x,acq_time_x,satellite,instrument,...,sys.sunset,timezone,id,name,cod,acq_year_y,acq_month_y,acq_day_y,acq_hour_y,acq_minute_y
0,THA,16.25453,102.07751,310.48,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,1743161021,25200,1611407,Chaiyaphum,200,2025,3,28,1,6
1,THA,17.09465,101.71175,303.78,3.47,1.75,2025-03-27,19:21:00,Aqua,MODIS,...,1743160947,25200,1608269,Nong Bua Lamphu,200,2025,3,28,1,6
2,THA,16.25340,102.08438,309.56,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,1743161021,25200,1611407,Chaiyaphum,200,2025,3,28,1,6
3,THA,16.23932,102.07933,306.22,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,1743161021,25200,1611407,Chaiyaphum,200,2025,3,28,1,6
4,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,1743161510,25200,1607758,Phayao,200,2025,3,28,1,6


In [12]:
dfmerge[['latitude_x','longitude_x','near_lat','near_lon','latitude_y','longitude_y']]

,latitude_x,longitude_x,near_lat,near_lon,latitude_y,longitude_y
0,16.25453,102.07751,15.8068,102.0315,15.8068,102.0315
1,17.09465,101.71175,17.2218,102.4260,17.2218,102.4260
2,16.25340,102.08438,15.8068,102.0315,15.8068,102.0315
3,16.23932,102.07933,15.8068,102.0315,15.8068,102.0315
4,19.45405,100.56052,19.2149,100.2024,19.2149,100.2024
...,...,...,...,...,...,...
3275,18.07610,103.97359,18.3609,103.6464,18.3609,103.6464
3276,18.07452,103.96301,18.3609,103.6464,18.3609,103.6464
3277,15.73711,100.81437,15.7040,100.1372,15.7040,100.1372
3278,18.07610,103.97359,18.3609,103.6464,18.3609,103.6464


In [40]:
dfmerge.drop(["distance_meters", "near_lat","near_lon"], axis = 1, inplace=True)

In [41]:
dfmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3280 entries, 0 to 3279
Data columns (total 62 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   country_id           3280 non-null   object                      
 1   latitude_x           3280 non-null   float64                     
 2   longitude_x          3280 non-null   float64                     
 3   brightness           3280 non-null   float64                     
 4   scan                 3280 non-null   float64                     
 5   track                3280 non-null   float64                     
 6   acq_date_x           3280 non-null   object                      
 7   acq_time_x           3280 non-null   object                      
 8   satellite            3280 non-null   object                      
 9   instrument           3280 non-null   object                      
 10  confidence           3280 non-null  

In [42]:
dfmerge.drop(dfmerge.columns[[57,58,59,60,61]], axis=1, inplace=True)

In [43]:
dfmerge.info()
dfmerge.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3280 entries, 0 to 3279
Data columns (total 57 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   country_id           3280 non-null   object                      
 1   latitude_x           3280 non-null   float64                     
 2   longitude_x          3280 non-null   float64                     
 3   brightness           3280 non-null   float64                     
 4   scan                 3280 non-null   float64                     
 5   track                3280 non-null   float64                     
 6   acq_date_x           3280 non-null   object                      
 7   acq_time_x           3280 non-null   object                      
 8   satellite            3280 non-null   object                      
 9   instrument           3280 non-null   object                      
 10  confidence           3280 non-null  

,country_id,latitude_x,longitude_x,brightness,scan,track,acq_date_x,acq_time_x,satellite,instrument,...,acq_time_y,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,timezone,id,name,cod
0,THA,16.25453,102.07751,310.48,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,01:06:50,2.0,2073065.0,TH,1743117005,1743161021,25200,1611407,Chaiyaphum,200
1,THA,17.09465,101.71175,303.78,3.47,1.75,2025-03-27,19:21:00,Aqua,MODIS,...,01:06:57,2.0,2034106.0,TH,1743116890,1743160947,25200,1608269,Nong Bua Lamphu,200
2,THA,16.25340,102.08438,309.56,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,01:06:50,2.0,2073065.0,TH,1743117005,1743161021,25200,1611407,Chaiyaphum,200
3,THA,16.23932,102.07933,306.22,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,01:06:50,2.0,2073065.0,TH,1743117005,1743161021,25200,1611407,Chaiyaphum,200
4,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,01:06:53,NaN,NaN,TH,1743117394,1743161510,25200,1607758,Phayao,200


export to csv

In [44]:
dfmerge.to_csv('testmerge.csv')

In [3]:
duplicate = df_firms[df_firms.duplicated()]
 
print("Duplicate Rows :")
 
# Print the resultant Dataframe
duplicate

Duplicate Rows :


,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
1550,THA,16.97863,103.59332,309.77,1.81,1.32,2025-04-15,14:44:00,Terra,MODIS,...,280.14,22.16,N,2025-04-15 14:44:00,2025-04-15 21:44:00+07:00,2025,4,15,21,44
1556,THA,14.80767,104.05650,322.02,2.10,1.41,2025-04-16,07:07:00,Aqua,MODIS,...,296.52,21.38,D,2025-04-16 07:07:00,2025-04-16 14:07:00+07:00,2025,4,16,14,7
1557,THA,15.37804,102.58728,335.43,2.66,1.57,2025-04-16,07:07:00,Aqua,MODIS,...,297.41,95.30,D,2025-04-16 07:07:00,2025-04-16 14:07:00+07:00,2025,4,16,14,7
1558,THA,15.38506,102.57905,325.31,2.66,1.57,2025-04-16,07:07:00,Aqua,MODIS,...,296.91,39.74,D,2025-04-16 07:07:00,2025-04-16 14:07:00+07:00,2025,4,16,14,7
1559,THA,15.80613,100.30142,322.69,3.92,1.84,2025-04-16,07:07:00,Aqua,MODIS,...,290.93,70.61,D,2025-04-16 07:07:00,2025-04-16 14:07:00+07:00,2025,4,16,14,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,THA,18.07452,103.96301,309.65,1.16,1.07,2025-05-04,14:10:00.000000,Terra,MODIS,...,292.27,8.06,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3275,THA,18.07610,103.97359,316.76,1.15,1.07,2025-05-04,14:10:00.000000,Terra,MODIS,...,292.40,14.99,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3276,THA,19.70258,98.09076,302.77,2.76,1.59,2025-05-04,14:10:00.000000,Terra,MODIS,...,291.74,16.23,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3278,THA,17.54023,99.73300,302.68,1.75,1.30,2025-05-03,20:29:00.000000,Aqua,MODIS,...,290.33,7.54,N,2025-05-03 20:29:00,2025-05-04 03:29:00+07:00,2025,5,4,3,29


In [5]:
unique_df = df_firms.drop_duplicates()
display(unique_df)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,297.41,28.27,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
1,THA,19.45886,100.53445,316.00,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,298.49,43.84,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
2,THA,19.46471,100.54360,315.08,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,298.54,39.24,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
3,THA,19.47333,98.03783,309.05,1.75,1.30,2025-03-28,03:28:00,Terra,MODIS,...,296.34,12.68,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
4,THA,14.07963,100.65222,321.68,3.57,1.77,2025-03-28,03:30:00,Terra,MODIS,...,297.11,64.02,D,2025-03-28 03:30:00,2025-03-28 10:30:00+07:00,2025,3,28,10,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3263,THA,15.73711,100.81437,300.97,2.05,1.39,2025-05-04,14:10:00.000000,Terra,MODIS,...,272.10,13.07,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3264,THA,18.07452,103.96301,309.65,1.16,1.07,2025-05-04,14:10:00.000000,Terra,MODIS,...,292.27,8.06,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3265,THA,18.07610,103.97359,316.76,1.15,1.07,2025-05-04,14:10:00.000000,Terra,MODIS,...,292.40,14.99,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3266,THA,19.70258,98.09076,302.77,2.76,1.59,2025-05-04,14:10:00.000000,Terra,MODIS,...,291.74,16.23,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10


In [6]:
df_firms

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,297.41,28.27,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
1,THA,19.45886,100.53445,316.00,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,298.49,43.84,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
2,THA,19.46471,100.54360,315.08,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,298.54,39.24,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
3,THA,19.47333,98.03783,309.05,1.75,1.30,2025-03-28,03:28:00,Terra,MODIS,...,296.34,12.68,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
4,THA,14.07963,100.65222,321.68,3.57,1.77,2025-03-28,03:30:00,Terra,MODIS,...,297.11,64.02,D,2025-03-28 03:30:00,2025-03-28 10:30:00+07:00,2025,3,28,10,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,THA,18.07610,103.97359,316.76,1.15,1.07,2025-05-04,14:10:00.000000,Terra,MODIS,...,292.40,14.99,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3276,THA,19.70258,98.09076,302.77,2.76,1.59,2025-05-04,14:10:00.000000,Terra,MODIS,...,291.74,16.23,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
3277,THA,17.54023,99.73300,302.68,1.75,1.30,2025-05-03,20:29:00.000000,Aqua,MODIS,...,290.33,7.54,N,2025-05-03 20:29:00,2025-05-04 03:29:00+07:00,2025,5,4,3,29
3278,THA,17.54023,99.73300,302.68,1.75,1.30,2025-05-03,20:29:00.000000,Aqua,MODIS,...,290.33,7.54,N,2025-05-03 20:29:00,2025-05-04 03:29:00+07:00,2025,5,4,3,29
